### CS4622 Mini Kaggle Competition - Classification
#### Author: Sabine Hollatz

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn import preprocessing, metrics
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

import seaborn as sns

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


### Loading the Data and Preprocessiong

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/stock_XY_train.csv')
df_train.head()

,Unnamed: 0,Ticker,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",...,Payables Turnover,Inventory Turnover,ROE,Capex per Share,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Growth (per Share),5Y Operating CF Growth (per Share),3Y Operating CF Growth (per Share),10Y Net Income Growth (per Share),5Y Net Income Growth (per Share),3Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),5Y Shareholders Equity Growth (per Share),3Y Shareholders Equity Growth (per Share),10Y Dividend per Share Growth (per Share),5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,Yr,Buy
0,4685,MGEN,2.538000e+06,-0.4125,2.472000e+06,6.600000e+04,1002000.0,10251000.0,1.125300e+07,-1.118700e+07,141000.0,-1.132800e+07,0.0,0.0,0.0,-1.132800e+07,0.0,-1.132800e+07,-21.02,-21.02,709385.0,539460.0,0.000,0.0260,-4.335,-4.4078,-4.463,-2.7664,-1.100300e+07,-1.118700e+07,-1.132800e+07,-4.4634,-4.4634,10832000.0,0.0,10832000.0,3.940000e+05,1.870000e+05,1.173400e+07,1.153000e+06,...,0.8547,13.8689,-1.0882,-0.2280,-0.9308,-0.9916,-0.6291,-0.7060,0.5994,0.5994,2.5853,2.5853,0.0000,-2.1170,-1.8322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.6379,0.0447,0.5950,-0.5153,0.0000,1.8876,0.3534,Healthcare,15,0
1,3142,SO,1.846700e+10,0.0808,1.201200e+10,6.455000e+09,0.0,0.0,1.945000e+09,4.510000e+09,835000000.0,3.008000e+09,977000000.0,0.0,0.0,2.031000e+09,68000000.0,1.963000e+09,2.19,2.18,899812716.0,897000000.0,2.083,0.3495,0.332,0.2081,0.106,0.0250,6.136000e+09,3.843000e+09,2.031000e+09,0.1629,0.1100,710000000.0,0.0,710000000.0,1.947000e+09,1.969000e+09,5.864000e+09,5.486800e+10,...,2.7060,8.6557,0.0971,-5.9677,0.0188,0.1360,0.0169,0.1940,0.1649,0.1658,0.0228,0.0227,0.0348,-0.0463,-0.0906,0.0264,0.0078,-0.0003,0.0592,0.0957,-0.0200,0.0089,0.0106,-0.0507,0.0531,0.0354,0.0302,0.0394,0.0376,0.0361,0.0704,-0.1432,0.0881,0.0352,0.0636,0.0000,0.0000,Utilities,14,1
2,15877,ONDK,3.509500e+08,0.2047,1.991250e+08,1.518250e+08,53392000.0,94702000.0,1.661700e+08,-1.434500e+07,0.0,-1.153400e+07,0.0,-2811000.0,0.0,-1.153400e+07,0.0,-1.153400e+07,-0.16,-0.16,73635045.0,72890313.0,0.000,0.4326,-0.005,-0.0329,-0.033,0.5868,-1.583000e+06,-1.153400e+07,-1.434500e+07,-0.0329,-0.0329,114824000.0,0.0,114824000.0,0.000000e+00,0.000000e+00,9.586050e+08,2.357200e+07,...,22.0871,0.0000,-0.0440,-0.0584,0.3994,0.8610,0.8322,0.8610,0.8632,0.8632,0.0276,0.0276,0.0000,0.5657,0.6748,NaN,-0.0226,0.1698,NaN,0.0596,0.1367,NaN,0.0000,0.0000,NaN,NaN,-0.1527,NaN,0.0000,0.0000,0.0000,0.0000,-0.0639,-0.0175,-0.0826,-0.0935,-0.1790,Financial Services,17,0
3,6977,FFNW,3.392500e+07,-0.0320,0.000000e+00,3.392500e+07,0.0,19533000.0,1.987800e+07,1.404700e+07,0.0,1.404700e+07,4887000.0,0.0,0.0,9.160000e+06,0.0,9.160000e+06,0.67,0.67,14107868.0,13528393.0,0.240,1.0000,0.438,0.4141,0.270,0.4402,1.485600e+07,1.404700e+07,9.160000e+06,0.4141,0.2700,105711000.0,0.0,105711000.0,0.000000e+00,0.00

In [4]:
df_train = df_train.drop(['Unnamed: 0', 'Ticker', 'Yr'], axis=1)
df_train.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,Long-term investments,...,Receivables Turnover,Payables Turnover,Inventory Turnover,ROE,Capex per Share,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Growth (per Share),5Y Operating CF Growth (per Share),3Y Operating CF Growth (per Share),10Y Net Income Growth (per Share),5Y Net Income Growth (per Share),3Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),5Y Shareholders Equity Growth (per Share),3Y Shareholders Equity Growth (per Share),10Y Dividend per Share Growth (per Share),5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,Buy
0,2.538000e+06,-0.4125,2.472000e+06,6.600000e+04,1002000.0,10251000.0,1.125300e+07,-1.118700e+07,141000.0,-1.132800e+07,0.0,0.0,0.0,-1.132800e+07,0.0,-1.132800e+07,-21.02,-21.02,709385.0,539460.0,0.000,0.0260,-4.335,-4.4078,-4.463,-2.7664,-1.100300e+07,-1.118700e+07,-1.132800e+07,-4.4634,-4.4634,10832000.0,0.0,10832000.0,3.940000e+05,1.870000e+05,1.173400e+07,1.153000e+06,0.000000e+00,0.0,...,3.4251,0.8547,13.8689,-1.0882,-0.2280,-0.9308,-0.9916,-0.6291,-0.7060,0.5994,0.5994,2.5853,2.5853,0.0000,-2.1170,-1.8322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.6379,0.0447,0.5950,-0.5153,0.0000,1.8876,0.3534,Healthcare,0
1,1.846700e+10,0.0808,1.201200e+10,6.455000e+09,0.0,0.0,1.945000e+09,4.510000e+09,835000000.0,3.008000e+09,977000000.0,0.0,0.0,2.031000e+09,68000000.0,1.963000e+09,2.19,2.18,899812716.0,897000000.0,2.083,0.3495,0.332,0.2081,0.106,0.0250,6.136000e+09,3.843000e+09,2.031000e+09,0.1629,0.1100,710000000.0,0.0,710000000.0,1.947000e+09,1.969000e+09,5.864000e+09,5.486800e+10,0.000000e+00,743000000.0,...,9.8072,2.7060,8.6557,0.0971,-5.9677,0.0188,0.1360,0.0169,0.1940,0.1649,0.1658,0.0228,0.0227,0.0348,-0.0463,-0.0906,0.0264,0.0078,-0.0003,0.0592,0.0957,-0.0200,0.0089,0.0106,-0.0507,0.0531,0.0354,0.0302,0.0394,0.0376,0.0361,0.0704,-0.1432,0.0881,0.0352,0.0636,0.0000,0.0000,Utilities,1
2,3.509500e+08,0.2047,1.991250e+08,1.518250e+08,53392000.0,94702000.0,1.661700e+08,-1.434500e+07,0.0,-1.153400e+07,0.0,-2811000.0,0.0,-1.153400e+07,0.0,-1.153400e+07,-0.16,-0.16,73635045.0,72890313.0,0.000,0.4326,-0.005,-0.0329,-0.033,0.5868,-1.583000e+06,-1.153400e+07,-1.434500e+07,-0.0329,-0.0329,114824000.0,0.0,114824000.0,0.000000e+00,0.000000e+00,9.586050e+08,2.357200e+07,0.000000e+00,0.0,...,0.0000,22.0871,0.0000,-0.0440,-0.0584,0.3994,0.8610,0.8322,0.8610,0.8632,0.8632,0.0276,0.0276,0.0000,0.5657,0.6748,NaN,-0.0226,0.1698,NaN,0.0596,0.1367,NaN,0.0000,0.0000,NaN,NaN,-0.1527,NaN,0.0000,0.0000,0.0000,0.0000,-0.0639,-0.0175,-0.0826,-0.0935,-0.1790,Financial Services,0
3,3.392500e+07,-0.0320,0.000000e+00,3.392500e+07,0.0,19533000.0,1.987800e+07,1.404700e+07,0.0,1.404700e+07,4887000.0,0.0,0.0,9.160000e+06,0.0,9.160000e+06,0.67,0.67,14107868.0,13528393.0,0.240,1.0000,0.438,0.4141,0.270,0.4402,1.485600e+07,

In [5]:
df_val = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/stock_X_test.csv')
df_val.head()

,Unnamed: 0,Unnamed: 0.1,Ticker,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,...,Receivables Turnover,Payables Turnover,Inventory Turnover,ROE,Capex per Share,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Growth (per Share),5Y Operating CF Growth (per Share),3Y Operating CF Growth (per Share),10Y Net Income Growth (per Share),5Y Net Income Growth (per Share),3Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),5Y Shareholders Equity Growth (per Share),3Y Shareholders Equity Growth (per Share),10Y Dividend per Share Growth (per Share),5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,Yr
0,0,17685,CMCSA,9.450700e+10,0.1115,0.000000e+00,9.450700e+10,0.000000e+00,6.482200e+10,7.549800e+10,1.900900e+10,3.542000e+09,1.511100e+10,3.380000e+09,131000000.0,0.000000e+00,1.173100e+10,0.0,1.173100e+10,2.56,2.53,4.549505e+09,4.549505e+09,0.917,1.0000,0.310,0.1974,0.124,0.1268,2.932900e+10,1.865300e+10,1.186200e+10,0.1599,0.1241,3.814000e+09,0.000000e+00,3.814000e+09,1.110400e+10,0.000000e+00,2.184800e+10,...,9.4801,3.6159,0.0000,0.1638,-2.7062,0.1115,0.0220,0.0550,-0.4840,-0.4700,-0.4674,-0.0266,-0.0305,0.9387,0.1428,0.2462,0.1327,0.1094,0.1107,0.1164,0.1456,0.1044,0.1928,0.1463,0.1579,0.0841,0.1019,0.1396,0.2558,0.1865,0.2348,0.2570,0.0000,0.3426,0.0722,0.7309,0.0000,0.1308,Consumer Cyclical,18
1,1,17686,KMI,1.414400e+10,0.0320,7.288000e+09,6.856000e+09,0.000000e+00,6.010000e+08,3.062000e+09,3.794000e+09,1.917000e+09,2.196000e+09,5.870000e+08,310000000.0,0.000000e+00,1.609000e+09,128000000.0,1.481000e+09,0.66,0.66,2.207018e+09,2.216000e+09,0.725,0.4847,0.453,0.2908,0.105,0.1498,6.410000e+09,4.113000e+09,1.919000e+09,0.1553,0.1138,3.280000e+09,0.000000e+00,3.280000e+09,1.498000e+09,3.850000e+08,5.722000e+09,...,9.6022,2.8248,34.9666,0.0478,-1.3195,0.0564,0.0405,0.0751,53.8519,65.0000,65.0000,-0.0063,-0.0063,0.4500,0.0961,0.3841,NaN,-0.1402,-0.0104,NaN,-0.1057,-0.0215,NaN,-0.0881,0.8445,NaN,0.0376,-0.0182,0.0000,-0.1421,-0.2785,0.0345,-0.0920,-0.0024,0.0076,-0.0137,0.0000,-0.1265,Energy,18
2,2,17687,INTC,7.084800e+10,0.1289,2.711100e+10,4.373700e+10,1.354300e+10,6.750000e+09,2.042100e+10,2.331600e+10,-1.260000e+08,2.331700e+10,2.264000e+09,0.0,0.000000e+00,2.105300e+10,0.0,2.105300e+10,4.57,4.48,4.564000e+09,4.611000e+09,1.200,0.6173,0.456,0.3273,0.297,0.2011,3.227600e+10,2.319100e+10,2.105300e+10,0.3291,0.2972,3.019000e+09,8.631000e+09,1.165000e+10,6.722000e+09,7.253000e+09,2.878700e+10,...,11.4929,5.5264,9.9534,0.2824,-3.2923,0.1187,0.1203,0.2917,1.1928,1.2402,1.2513,-0.0191,-0.0277,0.1132,0.3312,0.3793,0.0876,0.0770,0.0959,0.1271,0.0883,0.1675,0.1719,0.1872,0.2377,0.0876,0.0665,0.0787,0.0815,0.0592,0.0772,0.1989,0.0387,0.0382,0.1014,-0.0169,0.0390,-0.0942,Technology,18
3,3,17688,MU,3.039100e+10,0.4955,1.250000e+10,1.789100e+10,2.141000e+09,8.130000e+08,2.897000e+09,1.499400e+10,3.420000e+08,1.430300e+10,1.680000e+08,3000000.0,0.000000e+00,1.413500e+10,0

In [6]:
df_val = df_val.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Ticker', 'Yr'], axis=1)
df_val.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,Long-term investments,...,Days of Inventory on Hand,Receivables Turnover,Payables Turnover,Inventory Turnover,ROE,Capex per Share,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Growth (per Share),5Y Operating CF Growth (per Share),3Y Operating CF Growth (per Share),10Y Net Income Growth (per Share),5Y Net Income Growth (per Share),3Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),5Y Shareholders Equity Growth (per Share),3Y Shareholders Equity Growth (per Share),10Y Dividend per Share Growth (per Share),5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector
0,9.450700e+10,0.1115,0.000000e+00,9.450700e+10,0.000000e+00,6.482200e+10,7.549800e+10,1.900900e+10,3.542000e+09,1.511100e+10,3.380000e+09,131000000.0,0.000000e+00,1.173100e+10,0.0,1.173100e+10,2.56,2.53,4.549505e+09,4.549505e+09,0.917,1.0000,0.310,0.1974,0.124,0.1268,2.932900e+10,1.865300e+10,1.186200e+10,0.1599,0.1241,3.814000e+09,0.000000e+00,3.814000e+09,1.110400e+10,0.000000e+00,2.184800e+10,4.443700e+10,1.676230e+11,7.883000e+09,...,0.0000,9.4801,3.6159,0.0000,0.1638,-2.7062,0.1115,0.0220,0.0550,-0.4840,-0.4700,-0.4674,-0.0266,-0.0305,0.9387,0.1428,0.2462,0.1327,0.1094,0.1107,0.1164,0.1456,0.1044,0.1928,0.1463,0.1579,0.0841,0.1019,0.1396,0.2558,0.1865,0.2348,0.2570,0.0000,0.3426,0.0722,0.7309,0.0000,0.1308,Consumer Cyclical
1,1.414400e+10,0.0320,7.288000e+09,6.856000e+09,0.000000e+00,6.010000e+08,3.062000e+09,3.794000e+09,1.917000e+09,2.196000e+09,5.870000e+08,310000000.0,0.000000e+00,1.609000e+09,128000000.0,1.481000e+09,0.66,0.66,2.207018e+09,2.216000e+09,0.725,0.4847,0.453,0.2908,0.105,0.1498,6.410000e+09,4.113000e+09,1.919000e+09,0.1553,0.1138,3.280000e+09,0.000000e+00,3.280000e+09,1.498000e+09,3.850000e+08,5.722000e+09,3.789700e+10,2.484500e+10,7.481000e+09,...,-19.2817,9.6022,2.8248,34.9666,0.0478,-1.3195,0.0564,0.0405,0.0751,53.8519,65.0000,65.0000,-0.0063,-0.0063,0.4500,0.0961,0.3841,NaN,-0.1402,-0.0104,NaN,-0.1057,-0.0215,NaN,-0.0881,0.8445,NaN,0.0376,-0.0182,0.0000,-0.1421,-0.2785,0.0345,-0.0920,-0.0024,0.0076,-0.0137,0.0000,-0.1265,Energy
2,7.084800e+10,0.1289,2.711100e+10,4.373700e+10,1.354300e+10,6.750000e+09,2.042100e+10,2.331600e+10,-1.260000e+08,2.331700e+10,2.264000e+09,0.0,0.000000e+00,2.105300e+10,0.0,2.105300e+10,4.57,4.48,4.564000e+09,4.611000e+09,1.200,0.6173,0.456,0.3273,0.297,0.2011,3.227600e+10,2.319100e+10,2.105300e+10,0.3291,0.2972,3.019000e+09,8.631000e+09,1.165000e+10,6.722000e+09,7.253000e+09,2.878700e+10,4.897600e+10,3.634900e+10,9.430000e+09,...,-97.6484,11.4929,5.5264,9.9534,0.2824,-3.2923,0.1187,0.1203,0.2917,1.1928,1.2402,1.2513,-0.0191,-0.0277,0.1132,0.3312,0.3793,0.0876,0.0770,0.0959,0.1271,0.0883,0.1675,0.1719,0.1872,0.2377,0.0876,0.0665,0.0787,0.0815,0.0592,0.0772,0.1989,0.0387,0.0382,0.1014,-0.0169,0.0390,-0.0942,Technology
3,3.039100e+10

In [7]:
df_train.dtypes

Revenue                 float64
Revenue Growth          float64
Cost of Revenue         float64
Gross Profit            float64
R&D Expenses            float64
                         ...   
Debt Growth             float64
R&D Expense Growth      float64
SG&A Expenses Growth    float64
Sector                   object
Buy                       int64
Length: 223, dtype: object

In [0]:
df_train['Sector'] = df_train['Sector'].astype('category')

In [9]:
df_train.dtypes

Revenue                  float64
Revenue Growth           float64
Cost of Revenue          float64
Gross Profit             float64
R&D Expenses             float64
                          ...   
Debt Growth              float64
R&D Expense Growth       float64
SG&A Expenses Growth     float64
Sector                  category
Buy                        int64
Length: 223, dtype: object

In [10]:
df_train.isna().sum()

Revenue                  796
Revenue Growth          1379
Cost of Revenue         1103
Gross Profit             794
R&D Expenses            1353
                        ... 
Debt Growth             1649
R&D Expense Growth      1565
SG&A Expenses Growth    1531
Sector                     0
Buy                        0
Length: 223, dtype: int64

In [0]:
# split predictors and response variable
X_train = df_train.drop('Buy', axis=1)
y_train = df_train['Buy']

In [0]:
# get quantitative features to fill missing values with the feature mean. No missing values in qualitative 
# feature 'Sector'.
X_train_quant = X_train.drop('Sector', axis=1)
columns_quant = list(X_train_quant.columns.values)

X_train[columns_quant] = X_train[columns_quant].transform(lambda x: x.fillna(x.mean()))

In [13]:
X_train.isna().sum()

Revenue                        0
Revenue Growth                 0
Cost of Revenue                0
Gross Profit                   0
R&D Expenses                   0
                              ..
Book Value per Share Growth    0
Debt Growth                    0
R&D Expense Growth             0
SG&A Expenses Growth           0
Sector                         0
Length: 222, dtype: int64

In [14]:
X_train.isnull().sum()

Revenue                        0
Revenue Growth                 0
Cost of Revenue                0
Gross Profit                   0
R&D Expenses                   0
                              ..
Book Value per Share Growth    0
Debt Growth                    0
R&D Expense Growth             0
SG&A Expenses Growth           0
Sector                         0
Length: 222, dtype: int64

In [0]:
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,10))
# ax.boxplot(X_train[columns_quant[:56]])
# # ax[1].boxplot(X_train[columns_quant[56:111]], labels=columns_quant[56:111])
# # ax[2].boxplot(X_train[columns_quant[111:167]], labels=columns_quant[111:167])
# # ax[3].boxplot(X_train[columns_quant[167:]], labels=columns_quant[167:])
# plt.xticks(rotation=90);

In [0]:
def outlierCheck(df, attr):
    outlier_indices = []
    outlier_values = []
    inlier_indices = []
    q1 = df[attr].quantile(.25)
    q3 = df[attr].quantile(.75)
    iqr = q3 - q1
    
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    for index, row in df.iterrows():
        if row[attr] > upper_bound:
            outlier_indices.append(index)
            outlier_values.append(row[attr])
        elif row[attr] < lower_bound:
            outlier_indices.append(index)
            outlier_values.append(row[attr])
        else:
            inlier_indices.append(index)
    return inlier_indices, outlier_indices, outlier_values

In [0]:
# outliers = []
# for feature in columns_quant:
#     inlier, outlier, _ = outlierCheck(X_train, feature)
#     print("In train_X feature {} are {} outliers".format(feature, len(outlier)))
# print("")

In [18]:
# normalize quantitative features
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = X_train.copy(deep=True)

for attr in columns_quant:
    X_train_scaled[attr] = min_max_scaler.fit_transform(X_train_scaled[attr].values.reshape(-1,1))
X_train_scaled.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,Long-term investments,...,Days of Inventory on Hand,Receivables Turnover,Payables Turnover,Inventory Turnover,ROE,Capex per Share,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Growth (per Share),5Y Operating CF Growth (per Share),3Y Operating CF Growth (per Share),10Y Net Income Growth (per Share),5Y Net Income Growth (per Share),3Y Net Income Growth (per Share),10Y Shareholders Equity Growth (per Share),5Y Shareholders Equity Growth (per Share),3Y Shareholders Equity Growth (per Share),10Y Dividend per Share Growth (per Share),5Y Dividend per Share Growth (per Share),3Y Dividend per Share Growth (per Share),Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector
0,0.000346,0.000293,0.001927,0.026966,0.004875,0.000336,0.017701,0.109884,0.033930,0.179635,0.465028,0.197957,0.679831,0.084306,0.014960,0.084306,0.926951,0.938946,0.000007,0.000005,0.000000,0.946201,0.981371,0.978756,0.984548,0.920402,0.058629,0.073554,0.084306,0.977891,0.982357,0.000015,0.0,0.000011,9.493976e-07,4.379155e-07,0.000010,0.000001,0.000000,0.000057,...,0.996381,0.050264,0.050919,0.000634,0.00002,0.996972,0.000280,0.076135,0.027614,0.899005,0.759988,0.759988,0.000232,1.061264e-06,0.002677,0.164840,0.269007,0.371623,0.246853,0.023155,0.320676,0.273652,0.116187,0.445127,0.201829,0.121777,0.236164,0.131833,0.048295,0.552612,0.252466,0.184640,0.000020,0.004791,0.002503,0.360157,0.000193,0.002944,0.007499,Healthcare
1,0.010472,0.000305,0.009668,0.040556,0.004831,0.000280,0.023917,0.135588,0.059516,0.208035,0.465644,0.197957,0.679831,0.092298,0.046932,0.092032,0.926951,0.938946,0.008538,0.008511,0.002386,0.950318,0.981818,0.979261,0.984987,0.920723,0.083387,0.088813,0.092298,0.978396,0.982859,0.000963,0.0,0.000719,4.691566e-03,4.610993e-03,0.004829,0.059200,0.000000,0.000802,...,0.996265,0.050301,0.053166,0.000396,0.00002,0.996971,0.000282,0.076186,0.027660,0.899190,0.759833,0.759833,0.000066,3.027235e-07,0.002770,0.164899,0.269166,0.373644,0.242207,0.022255,0.329124,0.286893,0.107313,0.436470,0.192966,0.108317,0.242298,0.131048,0.047269,0.587597,0.260631,0.188146,0.000058,0.003930,0.001708,0.360207,0.000205,0.001723,0.006418,Utilities
2,0.000537,0.000308,0.002054,0.027286,0.007180,0.000790,0.018199,0.109866,0.033926,0.179633,0.465028,0.197606,0.679831,0.084305,0.014960,0.084305,0.926951,0.938946,0.000699,0.000692,0.000000,0.951376,0.981785,0.979234,0.984973,0.920788,0.058667,0.073553,0.084294,0.978375,0.982843,0.000156,0.0,0.000116,0.000000e+00,0.000000e+00,0.000789,0.000025,0.000000,0.000057,...,0.996481,0.050244,0.076690,0.000000,0.00002,0.996972,0.000283,0.076218,0.027719,0.899328,0.760082,0.760082,0.000066,3.041739e-07,0.002677,0.164916,0.269235,0.371623,0.234901,0.026042,0.320676,0.277440,0.124472,0.445127,0.190942,0.114102,0.236164,0.131833,0.038559,0.552612,0.251187,0.181590,0.000054,0.004586,0.001469,0.360203,0.000178,0.001662,0.005871,Financial Services
3,0.000363,0.

### Training Binary Classifiers

#### Stochastic Gradient Descent classifier

In [19]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=7)
sgd_clf.fit(X_train[columns_quant], y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=7, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [20]:
cross_val_score(sgd_clf, X_train[columns_quant], y_train, cv=5, scoring="accuracy")

array([0.51655897, 0.44870759, 0.4503231 , 0.44749596, 0.44040404])

In [21]:
y_train_pred = cross_val_predict(sgd_clf, X_train[columns_quant], y_train, cv=5)
metrics.confusion_matrix(y_train, y_train_pred)

array([[2541, 3437],
       [3239, 3162]])

In [22]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=7)
sgd_clf.fit(X_train_scaled[columns_quant], y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=7, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [23]:
cross_val_score(sgd_clf, X_train_scaled[columns_quant], y_train, cv=5, scoring="accuracy")

array([0.48344103, 0.51938611, 0.51857835, 0.48182553, 0.52282828])

In [24]:
log_reg = linear_model.LogisticRegressionCV(solver='lbfgs', max_iter=10000).fit(X_train_scaled[columns_quant], y_train)
y_hat_log_reg = cross_val_predict(log_reg, X_train_scaled[columns_quant], y_train, cv=5)
metrics.confusion_matrix(y_train, y_hat_log_reg)

array([[2183, 3795],
       [1619, 4782]])

In [25]:
cross_val_score(log_reg, X_train_scaled[columns_quant], y_train, cv=5, scoring="accuracy")

array([0.58037157, 0.55977383, 0.54644588, 0.56785137, 0.55878788])